#### 시험장 나누기
---
https://school.programmers.co.kr/learn/courses/30/lessons/81305

1. k개의 그룹으로 나누기
2. 각 그룹에서 인원의 합의 최대값을 최소화

1. 리프에서 부터 누적합을 계산해보자
2. 연결을 제거하면 부모노드의 누적 합에서 계속 빼주자 

3. 

1. 밑에서 부터 계산
2. 처음에는 리프 노드 수 만큼의 그룹에서
3. 점점 합쳐 가면서 그룹을 k개 까지 줄이기

## DP
포함 배제

### Parametric Search
https://blog.encrypted.gg/1003
- 각 그룹의 수를 x명으로 제한 할때 k 그룹이 되는지 판단
    - 리프에서 부터 합이 x 보다 크면 그룹 추가

In [45]:
import sys
sys.setrecursionlimit(10 ** 6)
class Node:
    def __init__(self, trafic):
        self.trafic = trafic
        self.parent = -1
        self.left = -1
        self.right = -1
    def show(self):
        print(self.trafic, f' {self.parent}- [{self.left}, {self.right}]')

class Tree:
    def __init__(self, k,  num, links):
        self.Nodes = [Node(trafic) for trafic in num]
        self.k = k
        for i in range(len(num)):
            left, right = links[i]
            if left != -1:
                self.Nodes[i].left = left
                self.Nodes[left].parent = i
            if right != -1:
                self.Nodes[i].right = right
                self.Nodes[right].parent = i

        self.root = 0
        while self.Nodes[self.root].parent != -1:
            self.root = self.Nodes[self.root].parent

    def show(self):
        for node in self.Nodes:
            node.show()

    def show_cut(self,ind):
        print(f'cut {ind}')

    def calc(self, ind, limit = 1):
        cur = self.Nodes[ind]
        node_trafic = cur.trafic

        if cur.left ==-1 and cur.right == -1:
            return node_trafic, 0

        elif cur.left != -1 and cur.right!=-1:
            left_trafic, add_group1, = self.calc(cur.left,limit)
            right_trafic, add_group2 = self.calc(cur.right,limit)


            if node_trafic+left_trafic+right_trafic > limit:
                min_ = min(left_trafic, right_trafic)
                
                if node_trafic+min_ > limit:

                    # self.show_cut(cur.left)
                    # self.show_cut(cur.right)
                    return node_trafic , add_group1+add_group2 + 2

                else:
                    # self.show_cut(cur.right if right_trafic> left_trafic else cur.left)
                    return node_trafic+min_ , add_group1+add_group2 +1
                
            else:
                return node_trafic+left_trafic+right_trafic, add_group1+add_group2

                
        elif cur.left != -1:
            left_trafic, add_group = self.calc(cur.left,limit)

            if node_trafic+left_trafic <= limit:
                return node_trafic+left_trafic,add_group
            else:
                # self.show_cut(cur.left)
                return node_trafic,add_group+1
        elif cur.right != -1:
            right_trafic, add_group = self.calc(cur.right,limit)

            if node_trafic+right_trafic <= limit:
                return node_trafic+right_trafic,add_group
            else:
                # self.show_cut(cur.right)
                return node_trafic,add_group+1

    

def solution(k, num, links):
    tree = Tree(k,num, links)
    lower , upper = max(num), sum(num)
    cur = (lower+upper)//2
    while lower!= upper:
        root_trafic, group = tree.calc(tree.root,cur)
        if root_trafic > cur or group+1 <=k:
            upper = cur
            cur = (lower+upper)//2
        else:
            lower = cur+1
            cur = (lower+upper)//2
        # print(cur, root_trafic, group+1)


    # root_trafic, group = tree.calc(9,40)
    # print(root_trafic, group)
    # return cur
    return cur

In [46]:
solution(	4, [6, 9, 7, 5], [[-1, -1], [-1, -1], [-1, 0], [2, 1]])

9

In [11]:
class Node:
    def __init__(self, trafic):
        self.trafic = trafic
        self.left = -1
        self.right = -1
    def show(self):
        print(self.trafic, f' - [{self.left}, {self.right}]')

class Tree:
    def __init__(self, k, num, links):
        self.Nodes = []
        self.k = k
        for i in range(len(num)):
            self.Nodes.append(Node(num[i]))
            left, right = links[i]
            if left != -1:
                self.Nodes[-1].left = left
            if right != -1:
                self.Nodes[-1].right = right

    def show(self):
        for node in self.Nodes:
            node.show()

    def calc(self, ind, limit, group_cnt = 1):

        if group_cnt >self.k:
            return 0,0,group_cnt
        cur = self.Nodes[ind]
        node_trafic = cur.trafic

        if cur.left != -1 and cur.right!=-1:

            left_trafic, add_group1, _ = self.calc(cur.left,limit, group_cnt)
            right_trafic, add_group2, _ = self.calc(cur.right,limit, group_cnt)
            group_cnt += add_group1
            group_cnt += add_group2
            if group_cnt >self.k:
                return 0,0,group_cnt

            if node_trafic+left_trafic+right_trafic <= limit:
                return node_trafic+left_trafic+right_trafic,0,group_cnt
            
            else:
                min_ = min(left_trafic, right_trafic)
                if node_trafic+min_ <= limit:
                    return node_trafic+min_ , 1, group_cnt
                else:
                    return node_trafic , 2,group_cnt
            
        elif cur.left != -1:
            left_trafic, add_group ,_ = self.calc(cur.left,limit, group_cnt)
            group_cnt += add_group
            if group_cnt >self.k:
                return 0,0,group_cnt

            if node_trafic+left_trafic <= limit:
                return node_trafic+left_trafic,0,group_cnt

        elif cur.right!=-1:
            right_trafic, add_group ,_ = self.calc(cur.right,limit, group_cnt)
            group_cnt += add_group
            if group_cnt >self.k:
                return 0,0,group_cnt

            if node_trafic+right_trafic <= limit:
                return node_trafic+right_trafic, 0, group_cnt
        
        if node_trafic <= limit:
            return node_trafic, 0, group_cnt
        
        else:
            return 0, 1, group_cnt
            

def solution(k, num, links):
    tree = Tree(k,num, links)
    sum_ = sum(num)
    print(tree.calc(9, k))
    answer = 0
    return answer